## **Sales Optimization and Dashboarding**

- This project leverage on the Cross Industry Standard Process for Data Mining (CRISP-DM) Approach to provide business intelligence for sales optimizations

### **Business Understanding**

- **Project Overview**

The client has gathered transactional data from the year 2019 but has not yet leveraged this information to inform their business strategy. The goal of this project is to analyze this data and deliver a comprehensive report that provides actionable insights. The client seeks to identify opportunities to increase sales and optimize operational efficiency.

- **Objective**

**Primary Goal**: Use the 2019 transactional data to uncover patterns, trends, and opportunities that can drive more sales and improve business processes.

**Secondary Goal**: Provide a clear roadmap based on data-driven insights that can support strategic decision-making and operational improvements.

- **Methodology**

To achieve these objectives, we will follow a systematic methodology that includes the following steps:

1. *Data Collection:* Retrieve transactional data for the entire year of 2019 from both CSV files (January to June) and the remote database (July to December).

2. *Data Preprocessing:* Clean and preprocess the data to handle missing values, outliers, and inconsistencies.

3. *Exploratory Data Analysis (EDA):* Conduct exploratory data analysis to gain insights into the distribution, trends, and patterns within the data.

4. *Statistical Analysis:* Perform statistical analysis to answer specific questions such as total revenue, seasonal trends, best-selling products, etc.

5. *Data Visualization:* Visualize the findings using charts, graphs, and tables to facilitate understanding and interpretation.

6. *Interpretation and Insights:* Provide actionable insights and recommendations based on the analysis to help the client make informed decisions.

7. *Report Generation:* Prepare a comprehensive report summarizing the analysis, findings, and recommendations for the client's reference.

- **Hypothesis Testing**

*Null Hypothesis, H<sub>0</sub>:* There is no relationship between Price Each and Quantity Ordered.


*Alternative Hypothesis, H<sub>1</sub>:* There is a relationship between Price Each and Quantity Ordered.

- **Analytical Questions**

1. How much money did we make this year? 

2. Can we identify any seasonality in the sales? 

3. What are our best and worst-selling products? 

4. How do sales compare to previous months or weeks? 

5. Which cities are our products delivered to most? 

6. How do product categories compare in revenue generated and quantities ordered? 

7. You are required to show additional details from your findings in your data. 

NB: Products with unit prices above $99.99 should be labeled high-level products otherwise they should be basic level.

### **Data Understanding**

- **Data Description**

*Features*

   - Order ID - This feature represents a unique identifier for each order.

   - Product - This feature represents the name or type of product ordered.

   - Quantity Ordered - This feature represents the quantity of each product ordered

   - Price Each - This feature represents the price of each product.

   - Order Date - This feature represents the date and time when the order was placed

   - Purchase Address - This feature represents the address where the purchase was made

*Numerical Features*

   - Quantity Ordered

   - Price Each

*Categorical Features*

   - Order ID

   - Product

   - Purchase Address

*Date features*

   - Order Date

#### **Importing relevant libraries**

In [1]:
# Data Manipulation and Handling
import pandas as pd
import numpy as np
import pyodbc
from dotenv import dotenv_values

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Analysis
import scipy.stats as stats

# Other Utilities
import os
import warnings
warnings.filterwarnings('ignore')

#### **Datasets from OneDrive**

In [ ]:
# File path for the OneDrive dataset for analysis and modelling
file_path1 = '../data/Sales_January_2019.csv'

# Load the file into the notebook
jan_sales = pd.read_csv(file_path1)
jan_sales.head()

#### **Datasets from the Remote Database**

In [2]:
# Loading environment variables from .env file
environment_variables = dotenv_values('.env')
 
# Getting the values for the credentials set in the .env file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")
 
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"
# Connecting to the server
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [3]:
# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA = 'dbo'
        """
# Read data from the SQL query result into a DataFrame using the established database connection
schema_df = pd.read_sql(db_query, connection)
 
#  Check whether data has been retrieved successfully to confirm successful connection to database
try:
    schema_df = pd.read_sql(db_query, connection)    
    print("Data retrieved successfully")
    print()
    print(schema_df)    
except Exception as e:
    print("Failed to retrieve data:", e)

Data retrieved successfully

  TABLE_CATALOG TABLE_SCHEMA            TABLE_NAME  TABLE_TYPE
0         dapDB          dbo       Sales_July_2019  BASE TABLE
1         dapDB          dbo     Sales_August_2019  BASE TABLE
2         dapDB          dbo  Sales_September_2019  BASE TABLE
3         dapDB          dbo    Sales_October_2019  BASE TABLE
4         dapDB          dbo   Sales_November_2019  BASE TABLE
5         dapDB          dbo   Sales_December_2019  BASE TABLE


In [11]:
# Querying the Database to determine the tables we are to use for analysis and modelling
query1 = "Select * from dbo.Sales_July_2019"

jul_sales = pd.read_sql(query1, connection)
jul_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,222910.0,Apple Airpods Headphones,1.0,150.00,2026-07-19 16:51:00.0000000,"389 South St, Atlanta, GA 30301"
1,222911.0,Flatscreen TV,1.0,300.00,2005-07-19 08:55:00.0000000,"590 4th St, Seattle, WA 98101"
2,222912.0,AA Batteries (4-pack),1.0,3.84,2029-07-19 12:41:00.0000000,"861 Hill St, Atlanta, GA 30301"
3,222913.0,AA Batteries (4-pack),1.0,3.84,2028-07-19 10:15:00.0000000,"190 Ridge St, Atlanta, GA 30301"
4,222914.0,AAA Batteries (4-pack),5.0,2.99,2031-07-19 02:13:00.0000000,"824 Forest St, Seattle, WA 98101"


In [10]:
query2 = "Select * from dbo.Sales_August_2019"

aug_sales = pd.read_sql(query2, connection)
aug_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,236670.0,Wired Headphones,2.0,11.990000,2031-08-19 22:21:00.0000000,"359 Spruce St, Seattle, WA 98101"
1,236671.0,Bose SoundSport Headphones,1.0,99.989998,2015-08-19 15:11:00.0000000,"492 Ridge St, Dallas, TX 75001"
2,236672.0,iPhone,1.0,700.000000,2006-08-19 14:40:00.0000000,"149 7th St, Portland, OR 97035"
3,236673.0,AA Batteries (4-pack),2.0,3.840000,2029-08-19 20:59:00.0000000,"631 2nd St, Los Angeles, CA 90001"
4,236674.0,AA Batteries (4-pack),2.0,3.840000,2015-08-19 19:53:00.0000000,"736 14th St, New York City, NY 10001"


In [12]:
query3 = "Select * from dbo.Sales_September_2019"

sep_sales = pd.read_sql(query3, connection)
sep_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,248151.0,AA Batteries (4-pack),4.0,3.840000,2017-09-19 14:44:00.0000000,"380 North St, Los Angeles, CA 90001"
1,248152.0,USB-C Charging Cable,2.0,11.950000,2029-09-19 10:19:00.0000000,"511 8th St, Austin, TX 73301"
2,248153.0,USB-C Charging Cable,1.0,11.950000,2016-09-19 17:48:00.0000000,"151 Johnson St, Los Angeles, CA 90001"
3,248154.0,27in FHD Monitor,1.0,149.990005,2027-09-19 07:52:00.0000000,"355 Hickory St, Seattle, WA 98101"
4,248155.0,USB-C Charging Cable,1.0,11.950000,2001-09-19 19:03:00.0000000,"125 5th St, Atlanta, GA 30301"


In [7]:
query4 = "Select * from dbo.Sales_October_2019"

oct_sales = pd.read_sql(query4, connection)
oct_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,259358.0,34in Ultrawide Monitor,1.0,379.989990,2028-10-19 10:56:00.0000000,"609 Cherry St, Dallas, TX 75001"
1,259359.0,27in 4K Gaming Monitor,1.0,389.989990,2028-10-19 17:26:00.0000000,"225 5th St, Los Angeles, CA 90001"
2,259360.0,AAA Batteries (4-pack),2.0,2.990000,2024-10-19 17:20:00.0000000,"967 12th St, New York City, NY 10001"
3,259361.0,27in FHD Monitor,1.0,149.990005,2014-10-19 22:26:00.0000000,"628 Jefferson St, New York City, NY 10001"
4,259362.0,Wired Headphones,1.0,11.990000,2007-10-19 16:10:00.0000000,"534 14th St, Los Angeles, CA 90001"


In [8]:
query5 = "Select * from dbo.Sales_November_2019"

nov_sales = pd.read_sql(query5, connection)
nov_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,278797.0,Wired Headphones,1.0,11.990000,2021-11-19 09:54:00.0000000,"46 Park St, New York City, NY 10001"
1,278798.0,USB-C Charging Cable,2.0,11.950000,2017-11-19 10:03:00.0000000,"962 Hickory St, Austin, TX 73301"
2,278799.0,Apple Airpods Headphones,1.0,150.000000,2019-11-19 14:56:00.0000000,"464 Cherry St, Los Angeles, CA 90001"
3,278800.0,27in FHD Monitor,1.0,149.990005,2025-11-19 22:24:00.0000000,"649 10th St, Seattle, WA 98101"
4,278801.0,Bose SoundSport Headphones,1.0,99.989998,2009-11-19 13:56:00.0000000,"522 Hill St, Boston, MA 02215"


In [9]:
query6 = "Select * from dbo.Sales_December_2019"

dec_sales = pd.read_sql(query6, connection)
dec_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,295665.0,Macbook Pro Laptop,1.0,1700.000000,2030-12-19 00:01:00.0000000,"136 Church St, New York City, NY 10001"
1,295666.0,LG Washing Machine,1.0,600.000000,2029-12-19 07:03:00.0000000,"562 2nd St, New York City, NY 10001"
2,295667.0,USB-C Charging Cable,1.0,11.950000,2012-12-19 18:21:00.0000000,"277 Main St, New York City, NY 10001"
3,295668.0,27in FHD Monitor,1.0,149.990005,2022-12-19 15:13:00.0000000,"410 6th St, San Francisco, CA 94016"
4,295669.0,USB-C Charging Cable,1.0,11.950000,2018-12-19 12:38:00.0000000,"43 Hill St, Atlanta, GA 30301"


#### **Merging all the 6 Dataframes (July - December, 2019) to become a Single Dataframe**

In [13]:
second_half_sales = pd.concat([jul_sales, aug_sales, sep_sales, oct_sales, nov_sales, dec_sales], ignore_index = True)

second_half_sales.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,222910.0,Apple Airpods Headphones,1.0,150.00,2026-07-19 16:51:00.0000000,"389 South St, Atlanta, GA 30301"
1,222911.0,Flatscreen TV,1.0,300.00,2005-07-19 08:55:00.0000000,"590 4th St, Seattle, WA 98101"
2,222912.0,AA Batteries (4-pack),1.0,3.84,2029-07-19 12:41:00.0000000,"861 Hill St, Atlanta, GA 30301"
3,222913.0,AA Batteries (4-pack),1.0,3.84,2028-07-19 10:15:00.0000000,"190 Ridge St, Atlanta, GA 30301"
4,222914.0,AAA Batteries (4-pack),5.0,2.99,2031-07-19 02:13:00.0000000,"824 Forest St, Seattle, WA 98101"


In [14]:
second_half_sales.tail()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
101220,319666.0,Lightning Charging Cable,1.0,14.950000,2011-12-19 20:58:00.0000000,"14 Madison St, San Francisco, CA 94016"
101221,319667.0,AA Batteries (4-pack),2.0,3.840000,2001-12-19 12:01:00.0000000,"549 Willow St, Los Angeles, CA 90001"
101222,319668.0,Vareebadd Phone,1.0,400.000000,2009-12-19 06:43:00.0000000,"273 Wilson St, Seattle, WA 98101"
101223,319669.0,Wired Headphones,1.0,11.990000,2003-12-19 10:39:00.0000000,"778 River St, Dallas, TX 75001"
101224,319670.0,Bose SoundSport Headphones,1.0,99.989998,2021-12-19 21:45:00.0000000,"747 Chestnut St, Los Angeles, CA 90001"
